# WHERE

As we have now covered SELECT and FROM and therefore we know how to correctly obtain columns of data from our tables, we now need to make this more applicable to the real world by filtering our data.

It is a very rare occurrence that we would ever return entire columns of data from our tables, especially in the modern world where it is commonplace to see tables in excess of a billion records.

When returning this level of data we have to consider the impact we are having on our SQL Server, the server itself, and the corporate network.

For example, let's say we have a table which is 100 GB in size and we decide to run the following:

In [ ]:
select *
from JupyterDatabase.dbo.mySuperBigTable

Admittedly that's not a big table, but I didn't want to put 1 billion rows of something into this example, therefore we need to use our imagination for the following scenario:

<u>SELECT everything from a 100GB table</u>

Let's imagine we are on a Windows Server and we run our query:

-   SSMS says "Can I have that big table please? Yes, really, all of it. Thanks"
-   SQL Server requests that the CPU help fetch it some data
-   SQL Server asks Windows Server to read data from the disks
-   Windows Server reads 100 GB data, passing it to SQL Server as it goes
-   SQL Server puts all this data into memory (if possible, let's presume it is)

At this point in time we now have upset disks, an upset CPU, and have filled the entire server's memory with data. But we're not done yet:

-   SQL Server streams 100GB data over the company network to your SSMS
-   SSMS stores the data in its memory on your desktop
-   Chances are your desktop crashes with an "out of memory" error

As you can see, we NEED to filter our data.

## The WHERE clause

In SQL we use the WHERE clause to filter data. This is incredibly efficient because SQL Server will filter the data on the server itself, potentially before even reading the data from disk and before sending anything across the network to SSMS. This reduces workload and resource consumption dramatically.

The WHERE clause copes with several types of filter, such as:

-   Equality
-   Inequality
-   Greater than / Less than
-   Between
-   Like
-   In / Not In

It can also cater for AND and OR in order to create combinations of filters and utilises special wildcard characters and patterns in order to allow for wider searches and pattern matching.

#### Equality and Inequality

This is the most basic form of the WHERE clause and which everyone will recognise immediately:

-   \=
-   <> or !=
-   < and <=
-   \> and >=

Utilising these does require a small amount of datatype knowledge (ie. is my data an integer, a string, a date etc) but in the most basic form and therefore it's a good starting point for getting used to WHERE.

Let's start with the most basic of all, the simple equality.

Firstly we shall have a look at a relatively small table which contains invented employee data from a fictitious company:

In [ ]:
select *
from JupyterDatabase.hr.employee

As this is a declarative language we code in the same way we would speak, for example, I wish to SELECT some data FROM this table WHERE the employeeID equals 1.

We write exactly that:

In [ ]:
select *
from JupyterDatabase.hr.employee
where employeeID = 1

We can see that will achieve our aim perfectly.  The WHERE clause follows the SELECT and FROM and specifies the filter we wish to see applied.

For integers and decimals this is simple as we simply write the value upon which we wish it to filter, but for string values we need a special tweak:

In [ ]:
select *
from JupyterDatabase.hr.employee
where lastName = 'Urquhart'

For a string or date we need to wrap our search terms in single quotes.

In the SQL language we interpret any non-numeric set of characters as the name of a column or a keyword.  Therefore if we wish to tell SQL Server that we are writing a string or we are defining a date we need to utilise the single quotes at the front and end of the value, as seen above.

In regard to dates, we would do this as follows:

In [ ]:
select *
from JupyterDatabase.hr.employee
where dob = '1979-02-27'

It's worth noting that the above date format is generally accepted as the norm, but this is not the exclusive way to write a date.  SQL Server will interpret many different formats of strings as dates and run them without issue.  Therefore, although the above is considered the norm, you are free to code with whatever format makes you feel most comfortable.

In [ ]:
select *
from JupyterDatabase.hr.employee
where dob = '27 Feb 1979'

In [ ]:
select *
from JupyterDatabase.hr.employee
where dob = '19790227'

In [ ]:
select *
from JupyterDatabase.hr.employee
where dob = '1979/02/27'

Inequality is done in exactly the same way as equality with the same overall syntax and form, just utilising a different symbol.

In SQL we would use the greater than and less than symbols: <>

In T-SQL we are also allowed to use a more literal not equal by typing !=.  However, although this is allowed I would recommend using the SQL standard version.

Let's give these a go and verify that they work:

In [ ]:
select *
from JupyterDatabase.hr.employee
where title <> 'Mr'

In [ ]:
select *
from JupyterDatabase.hr.employee
where title != 'Mr'

As we can see from the above, the two queries give the same results and are equivalent to each other. They also display the expected and correct results in that they do not return any records who have a title of 'Mr'.

In regard to greater than and less than, these work in exactly the same way and are very easy to code using exactly the same syntax as above.

If we want all employees where their employeeID is less than or equal to 3:

In [ ]:
select *
from JupyterDatabase.hr.employee
where employeeID <= 3

Now let's return all employees where their employeeID is greater than 3:

In [ ]:
select *
from JupyterDatabase.hr.employee
where employeeID > 3

As you can see, these are quite self-explanatory and therefore both simple to use and easy to grasp.

These same inequalities can be easily applied to dates as well.

In the following we will look for all employees who were born (the dob field stands for Date of Birth) after the 1st January 2000:

In [ ]:
select *
from JupyterDatabase.hr.employee
where dob > '2000-01-01'

The only word of caution in regard to inequality comes from the use of inequality with characters (not that I would recommend using these with strings but just in case you come across this).

For example, let's see what happens when we run the following:

In [ ]:
select *
from JupyterDatabase.hr.employee
where lastName > 'U'

What you'll note is that we still get results in which the lastName of the employee begins with the letter U. The way this is interpreted is very literally and SQL Server will see greater than U as anything that is greater than the single letter U when written alphabetically. Therefore, for example, anything beginning UA would be accepted.

As mentioned, I don't see why this would ever be useful or needed, but just in case you come across a use case, it's handy to know how it will be interpreted.

#### Other Operators

Beyond the simple equality and inequality we have other operators we can use to filter our results such as LIKE, IN, and BETWEEN. These can also be negated by the use of NOT, for example NOT IN.

These operators allow us to move beyond the single argument filter and select ranges of data.

##### LIKE

In the case of LIKE we are able to use a wildcard search, such as "find me any employees with a lastname which begins with 'Urq' but can have any characters you like after that".

The wildcard character we use in the SQL language is the percentage sign %. Therefore the query above would be written as follows:

In [ ]:
select *
from JupyterDatabase.hr.employee
where lastName like 'Urq%'

As you can see, we are using the wildcard to allow for any characters beyond the initial 'Urq'.

Utilising similar logic we can look for any strings which end with a set phrase or collection of characters, for example finding any employees where their firstName ends with 'vin':

In [ ]:
select *
from JupyterDatabase.hr.employee
where firstName like '%vin'

Taking this one step further we can look for any record in which the firstName has the character set 'ev' contained anywhere in it.  Translated, this means the firstName can begin with anything and end with anything, but must have 'ev' somewhere in the string:

In [ ]:
select *
from JupyterDatabase.hr.employee
where firstName like '%ev%'

But we're not quite done yet.  We can also use the wildcard symbol multiple times within a single search string.

Using this last example in the LIKE category we are going to search for any employee with a lastName that contains a 'u' and an 'h' in that order.  The lastName can start with anything, end with anything, and have any number of characters between the 'u' and 'h', but the 'u' and 'h' must be in the correct order:

In [ ]:
select *
from JupyterDatabase.hr.employee
where lastName like '%u%h%'

##### IN

Where LIKE is useful for basic patterns and searching on unknowns (I know it begins with 'X', but I'm not sure beyond that), if we want to filter on a specific and known set of data we use the keyword IN.

For example, let's say we wish to find all employees from our table with the names 'Kevin' or 'Diana'.  We can do this as follows:

In [ ]:
select *
from JupyterDatabase.hr.employee
where firstName in ('Kevin', 'Diana')

Or perhaps we know some employee numbers and wish to find out other information about them such as their first and last names:

In [ ]:
select employeeID, firstName, lastName
from JupyterDatabase.hr.employee
where employeeID in (5, 11, 39)

As you can see from the above, when filtering on a list of values we use the IN keyword, followed by a bracketed list of values which are separated by a comma.

##### BETWEEN

We can now easily write queries that search for patterns or reference lists of data, but what about ranges?  We may know the start and end of a range and want everything in 'between' those two values.

We could potentially write an IN statement, but we may not know every value in our range and, in some cases, even if we did know them the list of values could be incredibly large indeed.

Hence we can use BETWEEN.

In the example we are about to use, let's list all employees where their employeeID is between 1 and 10:

In [ ]:
select *
from JupyterDatabase.hr.employee
where employeeID between 1 and 10

One of the most common misconceptions around BETWEEN is whether it is inclusive or not and, as you can see by the above, it IS inclusive.  Requesting data between 1 and 10 will give you all records including both records with employeedIDs 1 and 10.

The reason that this can be contentious is when we use BETWEEN with dates.

Let's have a look at our employee with ID 121:

In [ ]:
select *
from JupyterDatabase.hr.employee
where employeeID = 121

We can see from the output that this employee record was created in January 2019, specifically 31st January 2019 at 18:20 (2019-01-31 18:20).

Now let's look at, what appears at first glance, a perfectly valid and logical query:

In [ ]:
select *
from JupyterDatabase.hr.employee
where createdDate between '2019-01-01' and '2019-01-31'

No results?

This is exactly where the misconception arises around the inclusivity of BETWEEN as an operator.  Surely we know that there is an employee who's record was created on Jan 31st and yet he doesn't appear in the results?

Therefore we'll presume it's not inclusive and write the following:

In [ ]:
select *
from JupyterDatabase.hr.employee
where createdDate between '2019-01-01' and '2019-02-01'

Now here is the troubling catch, we have now returned TWO records. One of them is the record we were looking for, but the other seems unexpected and seems to imply that BETWEEN is inclusive after all. So what on Earth has happened?

Well actually this is a hole that many people fall into (I've even seen this done incorrectly in banks and financial institutions) in which they do not realise that they are working with dates AND times.

As we are referring to a column which includes dates and times then taking our use of '2019-01-01', this should be correctly interpreted as '2019-01-01 00:00'.

For this reason it's easier to see that `BETWEEN '2019-01-01' and '2019-01-31'` actually stops at midnight at the very START of the 31st and therefore does not include anything with a time beyond that point. `BETWEEN '2019-01-01' and '2019-02-01'` includes anything up to and including midnight on the 1st February and therefore correctly returns the "unexpected" record we saw.

To ensure that you do not fall into this trap and are using dates and times, I would recommend you follow one of the following coding styles instead (note the latter uses AND which we will come across shortly, therefore feel free to make a note for the moment and revisit shortly if required):

In [ ]:
select *
from JupyterDatabase.hr.employee
where createdDate between '2019-01-01' and '2019-01-31 23:59:59.999'

In [ ]:
select *
from JupyterDatabase.hr.employee
where createdDate >= '2019-01-01'
and createdDate < '2019-02-01'

Ensuring you utilise one of the above will safeguard you against any troubling mishaps going forwards.

##### NOT

The last of our keywords in this section is NOT which is simply a negative keyword and can be combined with any of those keywords we have seen and dabbled with above.

In a nutshell, instead of selecting records where firstName is LIKE some pattern, we can specify where firstName is NOT LIKE some pattern.  Records that are IN a list can be negated by utilising NOT IN the same list.  And likewise NOT BETWEEN two dates, for example:

In [ ]:
select *
from JupyterDatabase.hr.employee
where firstName not like 'Ke%'

In [ ]:
select *
from JupyterDatabase.hr.employee
where employeeID not in (1, 2, 3, 4, 5, 6, 7, 8)

In [ ]:
select *
from FoodNStuff.hr.employee
where createdDate not between '2018-01-01' and '2018-12-31 23:59:59.999'

#### Combining Operators

I think we can all agree that it would be great if our data was so simple that a solitary filter would suffice in answering all our data requirements, but we know that in reality that is rarely ever the case. Therefore we need the ability to combine our operators and expand our WHERE clauses into something a little more flexible, complex, and detailed.

To this end the SQL language provides the AND and OR keywords for us to use.

##### AND

With AND we are stating that we wish all of our stated criteria to be met. If we are looking for data where our employeeIDs are less than or equal to 10 AND their date of birth is before 1980 then we are insisting that both criteria must be met and therefore, as the above description implies, we use AND.

Firstly let's get our employees where their IDs are less than or equal to 10:

In [ ]:
select *
from JupyterDatabase.hr.employee
where employeeID <= 10

Now we can write our query to obtain all employees with a date of birth less than 1980:

In [ ]:
select *
from JupyterDatabase.hr.employee
where dob < '1980-01-01'

So, in order to combine these into one single query and ensure that both criteria must be met, we can utilise AND:

In [ ]:
select *
from JupyterDatabase.hr.employee
where employeeID <= 10
and dob < '1980-01-01'

It's as simple as that.  It's literally just a way of extending your query to include more criteria without large amounts of additional code.

These can be as complex and intricate as required.  In the following example I am combining four criteria together in order to obtain any employees that have:
* EmployeeID IN the IDs 1 to 10
* firstName which starts with K (LIKE K)
* Date of Birth prior to 1980 (less than 1980)
* CreatedDate in 2018 (between 2018-01-01 and 2018-12-31 23:59:59.999)

In [ ]:
select *
from JupyterDatabase.hr.employee
where employeeID in (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
and firstName like 'K%'
and dob < '1980-01-01'
and createdDate between '2018-01-01' and '2018-12-31 23:59:59.999'

AND is great when we wish to endlessly narrow down our data to a specific set of cases where all of our criteria are met, but what about scenarios in which we may not wish all criteria to be met, but only some of them?

##### OR

We can use the OR keyword when we wish to suggest that "one or the other" criteria must be met in order to filter our records.

For example, we might say "I wish to find all employees where their firstName is like 'K' OR where their firstName is like 'D'". In this scenario we cannot use AND because the data we wish to see is mutually exclusive (a person cannot have a firstName beginning with two different letters), therefore we literally want to say "this criteria OR that criteria" and hence we utilise OR for this exact purpose:

In [ ]:
select *
from JupyterDatabase.hr.employee
where firstName like 'K%'
or firstName like 'D%'

However, we can hopefully all spot the slippery slope towards which we're heading with this and, if not, you soon will with the following example:

I wish to see all employees where
* firstName like 'K' OR firstName like 'D'
* managerID is less than 10

In [ ]:
select *
from JupyterDatabase.hr.employee
where firstName like 'K%'
or firstName like 'D%'
and managerID < 10

If you look at the result of the above you may well see that there's an issue.  We seem to have numerous records in which the managerID is larger than 10.  But we used AND, therefore how is this possible?

Well this comes about due to clarity of query.  The example I prefer to use in this circumstance is that of the "internet maths quizzes" that we all no doubt see appearing from time to time in which someone will post "What is the answer to the following: 2 + 2 - 2 + 2 * 2 - 2 / 2" and immediately fights break out between all and sundry as to the correct order of processing the operators.

In exactly the same way, SQL Server has a set of rules around how it processes the operators in the WHERE clause and, in this specific case, it has done it as follows:
* firstName LIKE 'K'
* firstName LIKE 'D' AND managerID < 10

Therefore, as with the maths above, to ensure we always get the information we want in a clear manner we need to add brackets for clarification:

In [ ]:
select *
from JupyterDatabase.hr.employee
where (firstName like 'K%'
or firstName like 'D%')
and managerID < 10

Therefore it's easy to combine filters using AND and OR, but be aware that you may need to include brackets to clarify your logic both to SQL Server, yourself, and to others who may view your code.

#### Pattern Matching

In our final section on the WHERE clause we are going to look at pattern matching in which we can solve intricate filtering needs in which basic operators such as equality and list based criteria are inadequate.

We can use SQL to look for generic or specific numerical characters, alphabetical characters, and alphanumerical characters.  We can also look for special characters if needed.

##### Alphabetical Character

In order to let SQL Server know that we are looking for a single alphabetical character we need to use special syntax involving the square brackets [ and ].

Inside these brackets we tell SQL Server the range of letter we are interested in.

For these demos we are going to keep looking at the same table but this time a new column, the National Insurance number of each employee (the US equivalent would be a Social Security Number).

In [ ]:
select employeeID, firstName, lastName, NINumber
from JupyterDatabase.hr.employee

Now, I have amended a couple of the values in this table so that we can utilise patterns in order to find specific records.

If you look at the above output you will see that employeeID 3 has an abnormal NI Number when compared to the standard pattern (2 letters, 10 numbers, 1 letter) and therefore it is this that we will be focussing on.

Firstly we will look at finding any records within the first 5 employeeIDs where the first character of the NI Number is alphabetical, ie. it fall somewhere between 'a' and 'z'. This, as mentioned above, should therefore eliminate employeeID 3.

The syntax we use to tell SQL Server that we want to look for such a range is by utilising square brackets. The square brackets denote one single character and the information we write inside the square brackets determines our search pattern.

Hence if we want the first character to be from 'a' to 'z' we write the following (remembering we are only looking at the first 5 employeeIDs):

In [ ]:
select employeeID, firstName, lastName, NINumber
from JupyterDatabase.hr.employee
where employeeID <= 5
and NINumber like '[a-z]%'

As you can see, this removes employeeID 4 from our resultset as you would expect as their NI Number does not start with a letter.

Therefore we can use exactly the same logic to find those records (just employeeID 4) who have the first character of their NI Number beginning with a number, ie. a character from 0 to 9:

In [ ]:
select employeeID, firstName, lastName, NINumber
from JupyterDatabase.hr.employee
where employeeID <= 5
and NINumber like '[0-9]%'

Note that we can use a negative (as we did with NOT previously) to look for something not in a range.

This is done using the carat symbol ^.

Therefore if we wish to look for an NI Number which does not begin with an alphabetical character we can do so as follows:

In [ ]:
select employeeID, firstName, lastName, NINumber
from JupyterDatabase.hr.employee
where employeeID <= 5
and NINumber like '[^a-z]%'

So far we have covered alphabetical and numberics, but what about special characters? This can be done in the same way, except the instead of specifying a range, such as 'a-z', we simply provide a list of characters we're interested in, similar to the IN keyword.

Let's look for an NI Number which contains (note contains, not begins with or ends with, but anywhere in the string, therefore we prefix and end the search criteria with a % sign) either an exclamation mark, comma, or minus sign:

In [ ]:
select employeeID, firstName, lastName, NINumber
from JupyterDatabase.hr.employee
where employeeID <= 5
and NINumber like '%[!,-]%'

Another useful function in pattern matching is to be able to specify an exact position in the string for your search.  For example, we might not be interested in the first character yet specifically want the second character to be an exclamation mark.

Using our existing method of % would not work in this case as this just translates as "any characters before the exclamation" whereas we specifically want the exclamation mark to be the second character in the string.

In SQL we can utilise the underscore for this purpose.  The underscore is similar in a way to the % sign in that it is a wildcard and stands for anything however, unlike the wildcard which can represent any number of characters, the underscore represents just one single character.

Therefore to answer our query (specifically the second character being an exclamation mark), we can use the underscore as follows:

In [ ]:
select employeeID, firstName, lastName, NINumber
from JupyterDatabase.hr.employee
where employeeID <= 5
and NINumber like '_[!]%'